Der folgende Code pertubiert Bilder. Der Algorithmus basiert auf der in dem Paper https://arxiv.org/abs/1710.10547 beschriebene Top-K Attacke.

Die Funktion pert nimmt das Ausgangsbild und die im Paper beschriebenen Parameter.<br>
Die Funktion nimmt jedoch ein weiteren Parameter "fix" entgegen. Falls dieser Parameter True ist sind die Top-K Features fix und werden nicht in jeder Iteration neu bestimmt.

Dieses Script nutzt die [crp2](crp2.yml) conda umgebung

In [ ]:
import torch
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision import models
from PIL import Image
import numpy as np
from zennit.attribution import Gradient
from zennit.composites import EpsilonPlusFlat

import os
from tqdm import trange

# This code doesnt work on mps, not tired on NVIDIA Cuda
device = torch.device("cpu")

imagenet_labels = models.VGG16_Weights.DEFAULT.meta["categories"]

weights=models.VGG16_BN_Weights.IMAGENET1K_V1.DEFAULT
model = models.vgg16_bn(weights=weights).to(device)
model.eval()

imagenet_mean = torch.tensor([0.485, 0.456, 0.406], device=device)
imagenet_std  = torch.tensor([0.229, 0.224, 0.225], device=device)
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
])

In [ ]:
def normalize(t):
    return (t - imagenet_mean[:, None, None]) / imagenet_std[:, None, None]

def unnormalize(t):
    return t * imagenet_std[:, None, None] + imagenet_mean[:, None, None]

def load_image(path, device):
    img = Image.open(path).convert("RGB")
    tensor = preprocess(img).unsqueeze(0).to(device)
    return tensor

def predict(model, inp_tensor):
    with torch.no_grad():
        logits = model(inp_tensor)
        probs = torch.nn.functional.softmax(logits, dim=1)
        pred = logits.argmax(dim=1).item()
    return pred, probs[0, pred].item()

In [ ]:
def topk_positions_from_relevance(relevance, k, reverse):
    a = relevance.detach().abs().sum(dim=1)  # (1,H,W)
    flat = a.view(-1)
    if k >= flat.numel():
        idx = torch.arange(flat.numel(), device=flat.device)
    else:
        _, idx = torch.topk(flat, k, largest=not reverse)
    H, W = a.shape[1], a.shape[2]
    ys = (idx // W).long()
    xs = (idx % W).long()
    return ys.cpu().numpy(), xs.cpu().numpy()

In [ ]:
def perturb_topk_features(model, image_tensor_0_1, k, alpha, epsilon, fix=True, maxRetries=10, device="cpu"):
    model.to(device).eval()
    img_norm = normalize(image_tensor_0_1.squeeze(0)).unsqueeze(0).to(device).clone().detach()
    img_orig_norm = img_norm.clone().detach()

    orig_pred, orig_conf = predict(model, img_norm)
    print(f"Original prediction: class {orig_pred}, confidence {orig_conf:.4f}")

    composite = EpsilonPlusFlat()
    attributor = Gradient(model, composite, create_graph=True)

    eps_norm = (epsilon / imagenet_std).view(3,1,1).to(device)
    alpha_norm = (alpha / imagenet_std).view(3,1,1).to(device)

    best_img = None
    best_score = -float("inf")
    best_img2 = None
    first_score = None
    last_one = 0
    was_last = False

    ys = None
    xs = None

    pbar = trange(maxRetries, desc="Pertubiere Image")
    for attempt in pbar:
        img_norm.requires_grad_(True)
        with attributor:
            output = model(img_norm)
            target = F.one_hot(torch.tensor([orig_pred], device=device), num_classes=output.shape[1]).float()
            _, relevance = attributor(img_norm, target)

        if ys is None or not fix:
            ys, xs = topk_positions_from_relevance(relevance, k, False)

        topk_mask = torch.zeros_like(relevance, device=device)
        for y, x in zip(ys, xs):
            topk_mask[0, :, y, x] = 1.0

        D = - (relevance * topk_mask).sum()
        grad_D, = torch.autograd.grad(D, img_norm, retain_graph=False)

        delta_norm = alpha_norm * torch.sign(grad_D)
        delta_norm = - delta_norm
        img_temp = img_norm + delta_norm
        delta = img_temp - img_orig_norm
        delta = torch.max(torch.min(delta, eps_norm), -eps_norm)
        perturbed_norm = delta + img_orig_norm

        perturbed_norm = (perturbed_norm).clamp(
            (0 - imagenet_mean[:, None, None].to(device)) / imagenet_std[:, None, None].to(device),
            (1 - imagenet_mean[:, None, None].to(device)) / imagenet_std[:, None, None].to(device)
        )
        
        pred_new, conf_new = predict(model, perturbed_norm)

        was_last = False
        if pred_new == orig_pred:
            with attributor:
                output = model(perturbed_norm)
                _, rel_new = attributor(perturbed_norm, target)
            score = rel_new.abs().sum(dim=1)[0, ys, xs].sum().item()

            best_img2 = perturbed_norm.detach()

            was_last = True
            if first_score is None:
                first_score = score 
                
            if score > best_score:
                best_score = score
                best_img = perturbed_norm.detach()
                best_img2 = None
                last_one = attempt
        else:
            print("Other Prediction")
                
        img_norm = perturbed_norm.detach().clone().requires_grad_(True)
        
    pbar.close()
    
    if best_img is not None:
        perturbed_unnorm = unnormalize(best_img.squeeze(0)).clamp(0.0,1.0).unsqueeze(0)
        
        delta_pixel = perturbed_unnorm - image_tensor_0_1
        linf = delta_pixel.abs().max().item()
        print(f"Final: gleiche Klasse, minimale Top-k Relevanz {best_score:.6f} von {first_score:.6f}, L_inf {linf:.6f}, last one at {last_one}")

        img_norm = None if was_last else img_norm
        return best_img, img_orig_norm, best_img2, img_norm, delta_pixel, True
    else:
        print("Keine erfolgreiche Perturbation gefunden.")
        return image_tensor_0_1, image_tensor_0_1, None, torch.zeros_like(image_tensor_0_1), False

In [ ]:
def pert(image_name, k, fix, maxRetries, alphaRawH, espilonRaw):
    image_path = f"images/{image_name}"
    img = load_image(image_path, device)
    img_cpu = img.detach().cpu()
    
    epsilon = epsilonRaw / 255.0 
    alpha = alphaRawH / 100.0 / 255.0

    perturbed, original, maxPert, maximum, delta, ok = perturb_topk_features(model, img, k=k, fix=fix, alpha=alpha, epsilon=epsilon, maxRetries=maxRetries, device=device)

    fixStr = "_F" if fix else ""
    result_folder = f"k{k}_retry{maxRetries}_alpha{alphaRawH}_epsilon{epsilonRaw}{fixStr}"
    print(result_folder)
    result_path = f"results/{image_name}/{result_folder}"

    isExist = os.path.exists(result_path)
    if not isExist:
       os.makedirs(result_path)

    resultsToBeSaved = {
        "perturbed": perturbed,
        "original": original
    }

    for name, norm_tensor in resultsToBeSaved.items():
        if norm_tensor is None:
            continue
        if name == "original":
            name = "../original"
        norm_tensor_cpu = norm_tensor.cpu()
        torch.save(norm_tensor_cpu,f"{result_path}/{name}.pt")
    
        pixel_tensor = unnormalize(norm_tensor_cpu.squeeze(0)).clamp(0, 1).detach()
        out = (pixel_tensor.permute(1,2,0).numpy() * 255.0).astype(np.uint8)
    
        img = Image.fromarray(out)
        img.save(f"{result_path}/{name}.png")


In [ ]:
k = 100
maxRetries = 100
alphaRawH = 6
epsilonRaw = 6

In [ ]:
imgNames = ["beagle1.jpg","dog.jpg","eel1.jpg","espresso.jpg","goldfish.jpg","lizard.jpg","plane.png","snail.png", "warplane.jpg"]
tryKs = [1, 10]
fixs = [True]

for i in imgNames:
    for fix in fixs:
        for k in tryKs:
            pert(i, k, fix, maxRetries, alphaRawH, epsilonRaw)

You can use a sound to get a notification when calculation is finished

In [ ]:
import IPython
# This audio-file is not within the repo
IPython.display.Audio("audio.mp3", autoplay=True)